## Understanding Dataset (Exploratory data analysis)

In [69]:
import pandas as pd
import numpy as np
from collections import Counter

In [70]:
df = pd.read_csv("sales.csv")

In [3]:
df.shape

In [2]:
df[df["billing_country"]=="China"].financial_status.value_counts()

In [1]:
L = []
for i in df["variant_sku"].unique()[1:]:
    if i is not None:
        L.append(i[0:2])
set(L)

## Group sku
### some of order doesn't have the sku record

In [47]:
df = pd.read_csv("sales.csv")

In [79]:
df.loc[df.variant_sku=="False","variant_sku"]= 0 #delete the "FALSE"

In [4]:
df["variant_sku"].value_counts()

In [81]:
df["grouped_product"] = df["variant_sku"].apply(lambda x: int(x[0:2]) if type(x)==str else "")

In [6]:
df["grouped_product"].value_counts()  # makes sense

## Create a column of product name

In [83]:
def create_name(group):
    if group == 21:
        name = "Women's +20"
    elif group == 41:
        name = "Women's +40"
    elif group == 40:
        name = "Men's +40"
    elif group == 20:
        name = "Men's +20"
    elif group == 22:
        name = "Petite Women's +20"
    elif group == 24:
        name = "Petite Women's +40"
    else:
        name = ""
    return name

In [84]:
df["product_name"] = df["grouped_product"].apply(create_name)

In [7]:
df["product_name"].value_counts()

### How often are customers buying

In [11]:
df["customer_id"].value_counts()

## Shipping city different from billing city

In [100]:
df_subset = df[df.billing_city != df.shipping_city][["billing_city","shipping_city"]] # different

In [12]:
df_subset.shape

In [14]:
sum(pd.isna(df_subset["billing_city"])==False)

In [13]:
sum(pd.isna(df_subset["shipping_city"])==False)

In [87]:
df_diff_ship_bill = df[(df.billing_city != df.shipping_city) & (pd.isna(df.billing_city)==False) &(pd.isna(df.shipping_city)==False)]

In [15]:
df_diff_ship_bill.shape

In [16]:
df_empty_ship_with_bill = df[(pd.isna(df.billing_city)==False) & (pd.isna(df.shipping_city)==True)]
df_empty_ship_with_bill[["billing_city","shipping_city"]]

In [17]:
# Pairs of different billing_city and shipping_city
L = []
i = []
for index,row in df_diff_ship_bill.iterrows():
    i = (row["billing_city"],row["shipping_city"])
    L.append(i)
new_L = map(tuple,L)
Counter(new_L).most_common()

In [95]:
df_diff_ship_bill["pair"]= L

/Users/xinyifang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
df_diff_ship_bill["pair"].value_counts() #correct

In [97]:
df_diff_ship_bill.to_csv("diff_ship_bill_city.csv",index=False)

## Orders - Whether there's any transaction

In [19]:
df["orders"].value_counts()

In [20]:
df["units_per_transaction"].value_counts()

In [21]:
sum(df["units_per_transaction"]>0)

In [22]:
df[(df["units_per_transaction"]>0) & (df["orders"]==1)].shape

In [23]:
df[df["orders"]==0].order_name  #xxx orders=0

In [24]:
df["financial_status"].value_counts()

In [25]:
#order = 0, financial status
print(df[(df["financial_status"]=="paid") & (df["orders"]==0)].shape[0],
df[(df["financial_status"]=="refunded") & (df["orders"]==0)].shape[0],
df[(df["financial_status"]=="partially_refunded") & (df["orders"]==0)].shape[0],
df[(df["financial_status"]=="partially_paid") & (df["orders"]==0)].shape[0],
df[(df["financial_status"]=="pending") & (df["orders"]==0)].shape[0])

In [26]:
#order = 1, financial status
print(df[(df["financial_status"]=="paid") & (df["orders"]==1)].shape[0],
df[(df["financial_status"]=="refunded") & (df["orders"]==1)].shape[0],
df[(df["financial_status"]=="partially_refunded") & (df["orders"]==1)].shape[0],
df[(df["financial_status"]=="partially_paid") & (df["orders"]==1)].shape[0],
df[(df["financial_status"]=="pending") & (df["orders"]==1)].shape[0])

In [27]:
df["cancelled"].value_counts()

In [28]:
df["sale_kind"].value_counts()

In [29]:
sum(df.net_sales!=0) #xxx orders has positive net sales, xxx not zero.

In [30]:
sum(df.total_sales>0)

## Returns - modify by financial status

In [31]:
df["returns"].value_counts()

In [32]:
df[df["returns"]>0]["returns"].value_counts()

In [33]:
# The order is paid but have record of return.
df_return_paid = df[(df.returns!=0) & (df.financial_status == "paid")][["financial_status","returns","net_sales","gross_sales","total_sales","discounts"]]
df_return_paid

In [34]:
sum(df_return_paid["net_sales"]==df_return_paid.["total_sales"])

In [35]:
df_return_paid.gross_sales.value_counts()

In [36]:
# The order is refunded and have record of return.
df_return_refunded = df[(df.returns<0) & (df.financial_status == "refunded")][["financial_status","returns","net_sales","gross_sales","total_sales","discounts"]]
df_return_refunded.head()

## Adjustment
some adjusted order is refunded, some is not. not very meaningful information

In [38]:
df.adjustment.value_counts()

In [39]:
df.financial_status.value_counts()

## financial status and sales kind

In [41]:
print(df[(df["financial_status"]=="paid") & (df["sale_kind"]=="order")].shape[0],
df[(df["financial_status"]=="refunded") & (df["sale_kind"]=="order")].shape[0],
df[(df["financial_status"]=="partially_refunded") & (df["sale_kind"]=="order")].shape[0],
df[(df["financial_status"]=="partially_paid") & (df["sale_kind"]=="order")].shape[0],
df[(df["financial_status"]=="pending") & (df["sale_kind"]=="order")].shape[0])

In [42]:
print(df[(df["financial_status"]=="paid") & (df["sale_kind"]=="return")].shape[0],
df[(df["financial_status"]=="refunded") & (df["sale_kind"]=="return")].shape[0],
df[(df["financial_status"]=="partially_refunded") & (df["sale_kind"]=="return")].shape[0],
df[(df["financial_status"]=="partially_paid") & (df["sale_kind"]=="return")].shape[0],
df[(df["financial_status"]=="pending") & (df["sale_kind"]=="return")].shape[0])

## Order Quantity

In [43]:
df["net_quantity"].value_counts() 

In [44]:
sum(df["net_quantity"]<0) #there's xxx refund order record.

In [45]:
df["ordered_item_quantity"].value_counts() #expect from 0, other numbers are right, consistent with above

In [46]:
sum(df["ordered_item_quantity"]>0) # there's transaction. orders = 1 and units_per_transaction > 0

In [47]:
#ordered_item_quantity = 0, financial status
print(df[(df["financial_status"]=="paid") & (df["ordered_item_quantity"]==0)].shape[0],
df[(df["financial_status"]=="refunded") & (df["ordered_item_quantity"]==0)].shape[0],
df[(df["financial_status"]=="partially_refunded") & (df["ordered_item_quantity"]==0)].shape[0],
df[(df["financial_status"]=="partially_paid") & (df["ordered_item_quantity"]==0)].shape[0],
df[(df["financial_status"]=="pending") & (df["ordered_item_quantity"]==0)].shape[0])
# same as order = 0. There's no transaction.

In [48]:
sum(df["financial_status"] == "refunded") # xxx order refunded and there's xxx record.

In [49]:
df.to_csv("modified_sales.csv",index=False)